# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name(s): Un Leng Wat
###### Python version: 3.6

In [307]:
# import libraries
import pandas as pd
import numpy as np
from math import log
import operator
import csv
import sys
import random 
from random import randint
from collections import Counter, defaultdict

In [2]:
def preprocess(csv_file):
    df = pd.read_csv(csv_file)
    return df

Supervised Naive Bayes Classifier 

In [3]:
def cal_class_prior_prob(classes):
    class_prior_prob = dict(Counter(classes))
    for key in class_prior_prob.keys():
        class_prior_prob[key] = class_prior_prob[key]/float(len(classes))
    return class_prior_prob

In [41]:
def cal_post_prob(df):
    unique_class = df[df.columns[-1]].unique()
    post_prob = {}
    header_list = list(df)
    att_name = header_list[-1]
    unique_val_list = []
    for col in df:
        #print(df[col].unique())
        unique_val_len = len(df[col].unique())
        unique_val_list.append(unique_val_len)
    unique_val_list = unique_val_list[:-1]
    print(unique_val_list)
    for Class in unique_class:
        post_prob[Class] = defaultdict(dict)   #create a dictionary of dictionaries
        
        # calculate the posteriors probability 
        new_df = df[df[att_name] == Class]
        del new_df[new_df.columns[-1]]
        count = 0
        for column in new_df:
            unique_attribute_val = unique_val_list[count]
            
            post_prob[Class][column] = (dict(Counter(new_df[column])))
            unique_val = new_df[column].unique()
            cleaned_unique_val = []
            missing_val_num = len(unique_val)
            for p in range(len(unique_val)):
                if unique_val[p] != '?':
                    cleaned_unique_val.append(unique_val[p])
                    missing_val_num -= 1
            if missing_val_num >0:
                del post_prob[Class][column]['?']
            # Laplace Smoothing
            col_len = float(len(new_df[column]))
            
            for value in cleaned_unique_val:
                post_prob[Class][column][value] = (1+float(post_prob[Class][column][value]))/(unique_attribute_val-missing_val_num+col_len)
            post_prob[Class][column]['unseen'] = 1/float(unique_attribute_val-missing_val_num+col_len)
            count+=1
    return post_prob

In [8]:
# This function should build a supervised NB model
def train_supervised(df):
    classes = df[df.columns[-1]]
    # Calculate the Prior Probabilities
    class_prior_prob = cal_class_prior_prob(classes)
    # Calculate the Posteriors Probabilities
    post_prob = cal_post_prob(df)
    # Output
    #print("Prior Probability: {}".format(class_prior_prob))
    print("Posteriors Probability: {}".format(post_prob))
    prob_list = [class_prior_prob, post_prob]
    return prob_list

In [9]:
# This function should predict the class for a set of instances, based on a trained model 
def predict_supervised(prob_list, test_list):
    result_list = []
    for test_instance in test_list:
        #print(test_instance)
        result_list.append(predict_supervised_helper(prob_list, test_instance))
    return result_list

In [10]:
def predict_supervised_helper(prob_list, test_instance):
    class_prior_prob = prob_list[0]
    post_prob = prob_list[1]
    prob_results = {}   
    unique_class = list(class_prior_prob.keys())
    
    for Class in unique_class:
        class_probability  = class_prior_prob[Class]

        for i in range(0,len(test_instance)):
            corresponding_attribute = list(post_prob[Class].keys())[i]
            attribute_prob_val = post_prob[Class][corresponding_attribute]
            
            if test_instance[i] in attribute_prob_val.keys():
                class_probability *= attribute_prob_val[test_instance[i]]
                #print("{}: {}".format(Class,attribute_prob_val[test_instance[i]]))
            else:
                class_probability *= attribute_prob_val['unseen']
                #print("{}: {}".format(Class,attribute_prob_val['unseen']))
        prob_results[Class] = class_probability
    #print(prob_results)
    result = max(prob_results.items(), key=operator.itemgetter(1))[0]
    return result

In [61]:
# This function should evaluate a set of predictions, in a supervised context 
# accuracy = the number of correct instances/ total number of instances
def evaluate_supervised(result_list, labels_list):
    error = []
    count = 0.0
    if len(result_list) != len(labels_list):
        print("Error: the number of predictions does not match with the number of labels")
        sys.exit()
    else:
        for i in range(0,len(result_list)):
            if result_list[i] == labels_list[i]:
                count += 1
            else:
                #print('mismatch: {} {}'.format(result_list[i], labels_list[i]))
                error.append(result_list[i])
    tempset = set(error)
    error_list = list(tempset)
    for err in error_list:
        print('{}: {}'.format(err, error.count(err)))
    
    accuracy = (count/len(result_list))*100
    return accuracy

In [308]:
#df = preprocess('2018S1-proj1_data\\sample.csv')               # acc = 100
#df = preprocess('2018S1-proj1_data\\mushroom-dos.csv')         # acc= 95.6420
#df = preprocess('2018S1-proj1_data\\breast-cancer-dos.csv')    # acc = 75.0877
#df = preprocess('2018S1-proj1_data\\hypothyroid-dos.csv')      # acc = 93.45351
df = preprocess('2018S1-proj1_data\\car-dos.csv')               # acc = 87.145 

prob_list = train_supervised(df)

test_df = df[df.columns[:-1]]
labels_df = df[df.columns[-1]]
test_list = test_df.values.tolist()
labels_list = labels_df.values.tolist()
result_list = predict_supervised(prob_list, test_list)
accuracy = evaluate_supervised(result_list, labels_list)

#print('result_list: ')
#print(result_list)
#print('labels_list: ')
#print(labels_list)
#print('accuracy: ')
print(accuracy)

#df.head(10)

[4, 4, 4, 3, 3, 3]
Posteriors Probability: {'unacc': defaultdict(<class 'dict'>, {'vhigh': {'vhigh': 0.2967848309975268, 'high': 0.2679307502061006, 'med': 0.22176422093981862, 'low': 0.213520197856554, 'unseen': 0.0008244023083264633}, 'vhigh.1': {'vhigh': 0.2967848309975268, 'high': 0.25968672712283597, 'med': 0.22176422093981862, 'low': 0.22176422093981862, 'unseen': 0.0008244023083264633}, '2': {'2': 0.26875515251442705, '3': 0.24814509480626545, '4': 0.24154987633965375, '5more': 0.24154987633965375, 'unseen': 0.0008244023083264633}, '2.1': {'2': 0.4752475247524752, '4': 0.25825082508250824, 'more': 0.2665016501650165, 'unseen': 0.0008250825082508251}, 'small': {'small': 0.3712871287128713, 'med': 0.32425742574257427, 'big': 0.30445544554455445, 'unseen': 0.0008250825082508251}, 'low': {'med': 0.2953795379537954, 'high': 0.22937293729372937, 'low': 0.4752475247524752, 'unseen': 0.0008250825082508251}}), 'acc': defaultdict(<class 'dict'>, {'vhigh': {'vhigh': 0.18814432989690721, 'h

In [ ]:
Unsupervised Naive Bayes Classifier 

In [297]:
def gen_random_prob():
    random_prob_list = []
    for i in range(df.shape[0]):
        random_prob_list.append([0]*unique_class_num) #class_num
    
    
    for row in range(df.shape[0]):
        
        max_prob = 1
        prob = random.uniform(0, max_prob)
        total_prob = 0
        total_prob += prob
        random_prob_list[row][0] = prob
        if unique_class_num > 2:
            for col in range(1, unique_class_num-1):
                prob = random.uniform(0, (1-total_prob))
                total_prob += prob
                random_prob_list[row][col] = prob 
        random_prob_list[row][-1] = 1-total_prob
    return random_prob_list

In [140]:
def un_cal_class_prob(df, unique_class_num, random_prob_list):
    prob_list = []
    class_count_list = []
    ans = 0
    for k in range(unique_class_num):
        for j in range(len(random_prob_list)):
            ans += random_prob_list[j][k]
        class_count_list.append(ans)
        ans = ans/float(df.shape[0])
        prob_list.append(ans)
    return [prob_list, class_count_list]  

In [141]:
def un_cal_post_prob(df, unique_class_num, random_prob_list, class_count_list):
    
    post_prob = {}
    
    header_list = list(df)
    header_list = header_list[:-1]
    
    
    # a list for the number of unique values in each attribute 
    unique_val_list = []
    for col in df:
        unique_val = list(df[col].unique())
        unique_val_list.append(unique_val)
    unique_val_list = unique_val_list[:-1]
    #print(unique_val_list)
    
    attribute_df = df[df.columns[:-1]]
    post_prob = {}
    
    for k in range(len(header_list)):
        post_prob[header_list[k]] = defaultdict(dict)
        for unique_val in unique_val_list[k]:
            post_prob[header_list[k]][unique_val] = defaultdict(dict)
            for j in range(unique_class_num):
                post_prob[header_list[k]][unique_val][j] = 0
                
        
        for i in range(len(df)):
            for j in range(unique_class_num):
                post_prob[header_list[k]][attribute_df.iloc[i][header_list[k]]][j] += random_prob_list[i][j]
                #post_prob[header_list[k]][attribute_df.iloc[i][header_list[k]]][j] /= class_count_list[j] #check number here 
            
                   
    return post_prob
    

In [193]:
# This function should build an unsupervised NB model 
def train_unsupervised(df, unique_class_num, random_prob_list):
    temp_list = un_cal_class_prob(df, unique_class_num, random_prob_list)
    prob_list = temp_list[0]
    class_count_list = temp_list[1]
    post_prob = un_cal_post_prob(df, unique_class_num, random_prob_list, class_count_list)
    #print(post_prob)
    #print(prob_list)
    #print(class_count_list)
    return [prob_list, post_prob]
    

In [194]:
def iteration(df, unique_class_num, prev_prob_list, test_data, header_list):
    temp_list = train_unsupervised(df, unique_class_num, prev_prob_list)
    prob_list = temp_list[0]
    post_prob = temp_list[1]
    results = predict_unsupervised(test_data, header_list, prob_list, post_prob, unique_class_num)
    return results

In [195]:
def predict_unsupervised(test_data, header_list, prior_prob, post_prob, unique_class_num):
    results = []
    temp = []
    ans = []
    for test_instance in test_data:
        result = predict_unsupervised_helper(test_instance, header_list, prior_prob, post_prob, unique_class_num)
        results.append(result)
    for i in range(len(results)):
        sum = 0
        for j in range(unique_class_num):
            sum += results[i][j]
        for n in range(unique_class_num):
            results[i][n] /= sum
    return results

In [196]:
# This function should predict the class distribution for a set of instances, based on a trained model
def predict_unsupervised_helper(test_instance, header_list, prior_prob, post_prob, unique_class_num):
    result = []
    total = 0.0

    for i in range(unique_class_num):
        ans = prior_prob[i]
        for j in range(len(header_list)):
            ans *= post_prob[header_list[j]][test_instance[j]][i]
        result.append(ans)

    return result

In [252]:
# This function should evaluate a set of predictions, in an unsupervised manner
def evaluate_unsupervised(labels_df, results, unique_class_num):
    
    predict = []
    for i in range(len(results)):
        max_val = max(results[i])
        for j in range(unique_class_num):
            if results[i][j] == max_val:
                predict.append(j)
    #print(predict)
                
    unique_labels = labels_df.unique()
    labels_1 = labels_df.replace(unique_labels, [0,1]).values.tolist()
    labels_2 = labels_df.replace(unique_labels, [1,0]).values.tolist()
    labels_list = [labels_1, labels_2]
    #print(labels_list)
    
    acc_list = []
    for label in labels_list:
        count = 0
        for i in range(len(predict)):
            if predict[i] == label[i]:
                count += 1

        accuracy = (count/len(label))*100
        acc_list.append(accuracy)
    
    max_acc = max(acc_list)
    for k in range(len(acc_list)):
        if acc_list[k] == max_acc:
            predict = k
            #predict = labels_list[k]

    return max_acc

In [286]:
#df = preprocess('2018S1-proj1_data\\sample2.csv') 
#df = preprocess('2018S1-proj1_data\\car-dos.csv')             
df = preprocess('2018S1-proj1_data\\hypothyroid-dos.csv')       
#df = preprocess('2018S1-proj1_data\\breast-cancer-dos.csv')   
#df = preprocess('2018S1-proj1_data\\mushroom-dos.csv')        
unique_class_num = len(df[df.columns[-1]].unique())
test_df = df[df.columns[:-1]]
test_data = test_df.values.tolist()  
header_list = list(df)[:-1]
labels_df = df[df.columns[-1]]

random_prob_list = gen_random_prob()
temp_list = train_unsupervised(df, unique_class_num, random_prob_list)
prior_prob = temp_list[0]
post_prob = temp_list[1]
prev_prob_list = predict_unsupervised(test_data, header_list, prior_prob, post_prob, unique_class_num)
# first iteration
prev_prob_list = iteration(df, unique_class_num, prev_prob_list, test_data, header_list)
print(prev_prob_list)

[[0.9983112160789457, 0.0016887839210542532], [0.9983733992610689, 0.00162660073893111], [0.9984720514374298, 0.001527948562570226], [0.9983733992610689, 0.00162660073893111], [0.9983948607562843, 0.001605139243715762], [0.998103718178024, 0.001896281821975969], [0.9984720514374298, 0.001527948562570226], [0.9988198669331627, 0.0011801330668372615], [0.9988766421869824, 0.0011233578130175402], [0.9984720514374298, 0.001527948562570226], [0.9984720514374298, 0.001527948562570226], [0.9984720514374298, 0.001527948562570226], [0.9985458639534047, 0.001454136046595335], [0.9984720514374298, 0.001527948562570226], [0.9983733992610689, 0.00162660073893111], [0.9984589165626833, 0.0015410834373167522], [0.9984720514374298, 0.001527948562570226], [0.9984720514374298, 0.001527948562570226], [0.9984720514374298, 0.001527948562570226], [0.9987436432478087, 0.0012563567521912183], [0.9984720514374298, 0.001527948562570226], [0.9983733992610689, 0.00162660073893111], [0.9984720514374298, 0.00152794

In [288]:
# second iteration
prev_prob_list = iteration(df, unique_class_num, prev_prob_list, test_data, header_list)
print(prev_prob_list)

[[1.0, 1.0954843304356195e-54], [1.0, 1.0987514690692765e-54], [1.0, 9.961574085023472e-55], [1.0, 1.0987514690692765e-54], [1.0, 1.0994415920995085e-54], [1.0, 1.2847304871453469e-54], [1.0, 9.961574085023472e-55], [1.0, 4.783559783936368e-55], [1.0, 4.334180689564828e-55], [1.0, 9.961574085023472e-55], [1.0, 9.961574085023472e-55], [1.0, 9.961574085023472e-55], [1.0, 8.216074239148719e-55], [1.0, 9.961574085023472e-55], [1.0, 1.0987514690692765e-54], [1.0, 6.147085794489258e-55], [1.0, 9.961574085023472e-55], [1.0, 9.961574085023472e-55], [1.0, 9.961574085023472e-55], [1.0, 5.2762176892131314e-55], [1.0, 9.961574085023472e-55], [1.0, 1.0987514690692765e-54], [1.0, 9.961574085023472e-55], [1.0, 3.7885888573794547e-56], [1.0, 9.961574085023472e-55], [1.0, 1.4170446337705596e-54], [1.0, 9.961574085023472e-55], [1.0, 9.961574085023472e-55], [1.0, 1.017349200246277e-54], [1.0, 9.961574085023472e-55], [1.0, 9.961574085023472e-55], [1.0, 4.766342136463395e-55], [1.0, 9.961574085023472e-55],

In [285]:
# third iteration
prev_prob_list = iteration(df, unique_class_num, prev_prob_list, test_data, header_list)
print(prev_prob_list)


[[2.1858603282112473e-157, 1.0], [5.386682737832116e-156, 1.0], [5.440420974796106e-154, 1.0], [1.6775228560058854e-154, 1.0], [6.135043917272113e-155, 1.0], [2.4319375603041386e-156, 1.0], [4.4686113997681836e-156, 1.0], [4.145957585288708e-156, 1.0], [5.8655896114610455e-154, 1.0], [5.301463317023244e-156, 1.0], [7.309289770622612e-157, 1.0], [3.599364092638692e-157, 1.0], [4.2018780033638516e-156, 1.0], [2.2710474004130417e-155, 1.0], [2.587664916792737e-156, 1.0], [2.671792864313357e-156, 1.0], [7.309289770622612e-157, 1.0], [7.3614574560381e-156, 1.0], [1.8605044300398391e-156, 1.0], [1.0214073061302301e-157, 1.0], [6.11310957619191e-156, 1.0], [1.6829553495205788e-156, 1.0], [2.4817585704565387e-156, 1.0], [5.14704093686109e-155, 1.0], [2.6512677692412772e-155, 1.0], [9.528331717057295e-157, 1.0], [2.4817585704565387e-156, 1.0], [4.1199893896170726e-156, 1.0], [1.9372569310883137e-154, 1.0], [3.8641432689577404e-157, 1.0], [8.463709450433569e-157, 1.0], [3.805899489006306e-156, 1

In [278]:
#print(prev_prob_list)
max_acc = evaluate_unsupervised(labels_df, prev_prob_list, unique_class_num)

print(max_acc)

51.80352086667488


Accuracy Result (%): 
cancer: 70.52631578947368        --> within 10%   --> good
hypo: 95.2561669829222           --> within 10%   --> good
mushroom: 51.80352086667488      --> over 10%     --> not good



Question:

Q1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.


Ans:

The No-Free-Lunch Theorem points out that there is always some bad distribution to fail a learning algorithm with a finite
number of training samples. That is to say, without any restrictions on the data distributions, the classifier would not be able to give us very useful theoretical guarantees or insights.

Since we start off from non-uniform random guesses, the distribution is going to be different every time we run the classifier. Sometimes when the distribution of the training is good then it converges at a better rate, and vice versa. When the convergence is slow, then the accuracy suffers.

It is a bad distribution when the initial distribution (the one that is randomly assigned) is close to be a uniform distribution. In this case, it will not converge fast enough in the iteration step to make the classifier work well. So when the distribution of the non uniform distribution is far from uniform, it will converge well and become a classifier that works good enough (i.e. within 10% Accuracy of the supervised NB). 

To illustrate this, let's take mushroom dataset and cancer dataset as an example. We will look into the cancer dataset first. In the initial random probability distribution, most of the distribution are 0.97 or 0.98 & 0.02. After the first iteration, the majority of the distribution becomes 0.99 & 0.01. In contrast, If we look into the first random distribution initialization the mushroom dataset, we will find out that the quite a lot of distribution is close to uniform (e.g. 0.45 & 0.55). After the first iteration, some of the distribution still stay close to uniform distribution (e.g. 0.4 & 0.6). Not until the third iteration that most of the distribution becomes 0.99 or 1. Thus, we can see the correlation between the convergence rate and the resulting accuracy. Very similiar pattern is also shown on the hypothyroid dataset.

Q2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.


Answer:

As we are using the training dataset as the test dateset, the issue of overfitting would not be the cause of the variation in accuracy in these four datasets. So the cause(s) must be the different nature of the fours datasets.

Below are the potential cause(s) of the variation in accuracy:

1) number of instances: The more training instances, the less likely the model is overfitted, the better the model is in terms of generalization. As the number of instances in these four vary, the accuracy will also vary.

2) the quality of data: the more accurate/realiable the data points are (i.e. data points are collected correctly and accurately) or the less missing values there are, the less noise there is in the dataset, and thus the higher the accuracy. As some datasets have more missing values or are less reliable than others, the accuracy vary. 

3) the number of attributes: the more attributes, the higher chance that not all of them are relevent/helpful and thus the higher chance of complicating the model but not improving, or even lower, the accuracy. As the number of attributes differ among the four datasets, the accuracy will be affected as well. In addition, the more attributes, the more likely the model would violate the assumption made in Naive Bayes, as explained below.

4) whether the dataset obeys the conditional independence assumption made in Naive Bayes: As Naive Bayes assumes that all the attributes are independent from each other, if the attributes of a dataset violates this assumption, then its accuracy will suffer. The variation in accuracy might be because some datasets violates this assumption more whilst others violates this assumption less. An example of a data set violates the conditional independence assumption is the car data set. The buying price and the confort level/safety level/number of doors are somewhat correlated in the reality, instead of being entirely independent from each other as we assume in Naive Bayes.

5) the class/label distribution: Since the label distributions are different among the four datasets, the accuracy will vary as well. That is because if the dataset is imbalanced then the classifier would give more weight to set of group which has higher number of records.


In [70]:
mushroom_df = preprocess('2018S1-proj1_data\\mushroom-dos.csv')         # acc= 95.6420
cancer_df = preprocess('2018S1-proj1_data\\breast-cancer-dos.csv')    # acc = 75.0877
hypo_df = preprocess('2018S1-proj1_data\\hypothyroid-dos.csv')      # acc = 95.2245
car_df = preprocess('2018S1-proj1_data\\car-dos.csv')               # acc = 87.145 
print('Number of attributes per dataset: ')
print('{}: {}'.format('mushroom', len(list(mushroom_df))))
print('{}: {}'.format('cancer', len(list(cancer_df))))
print('{}: {}'.format('hypo', len(list(hypo_df))))
print('{}: {}'.format('car', len(list(car_df))))
print('Number of instances per dataset: ')
print('{}: {}'.format('mushroom', len(mushroom_df)))
print('{}: {}'.format('cancer', len(cancer_df)))
print('{}: {}'.format('hypo', len(hypo_df)))
print('{}: {}'.format('car', len(car_df)))

Number of attributes per dataset: 
mushroom: 23
cancer: 10
hypo: 19
car: 7
Number of instances per dataset: 
mushroom: 8123
cancer: 285
hypo: 3162
car: 1727


Class distribution per dataset:
cancer: {'no-recurrence-events': 201, 'recurrence-events': 84};     mushroom: {'e': 4208, 'p': 3915};                  hypothyroid: {'negative': 3012, 'hypothyroid': 150};          car: {'unacc': 1209, 'acc': 384, 'good': 69, 'vgood': 65}

Error: (wrong prediction; the actual label is different from the predicted class)

cancer:
no-recurrence-events: 41
recurrence-events: 30

mushroom:
e: 334
p: 20

hypothyroid:
hypothyroid: 59
negative: 148

car:
good: 12
acc: 121
vgood: 2
unacc: 87

In [51]:
label_df = df[df.columns[-1]]
print(Counter(label_df.values.tolist()))

Counter({'unacc': 1209, 'acc': 384, 'good': 69, 'vgood': 65})


Surprising Results:

1. the accuracy of the mushroom dataset is surprisely high even though its class distribution is not skew at all, has a lot of attributes and has a lot of missing values. It is possibly because if we exclude the rows with missing values we would still have more than enough data to train a good performing model. Perhaps this is due to the imbalanced class distribution that the dataset has. However, although this classifier only has an error rate of around 5%, it is still not an excellent classifier as over 300 mushrooms which are poision are identify as edible and thus might cause danger for anyone who trusts the prediction. 

2. it is quite surprising that the hypothyroid dataset achieves such a high accuracy as it has 19 attributes. It is unlikely that they are completely independent from each other. But since our ultimate goal is to predict the most probable class, instead of a probability, it does not matter a lot if we inaccurately estimate the probability to some degree. 

3. even though three out of four datasets have missing values, it doesn't affect the accuracy too much as we still have enough instances to train a good model even if we discard the entire low of those that have missing value. This is one of the advantages of Naive Bayes - i.e. we don't need a lot of data to build a good model. 

4. the accuracy of the cancer dataset is relatively low compared to other datasets. As it only has a few missing data and a reasonable number of attributes, the result is quite surprising. One of the possible reasons for this might be the inbalanced class distribution. 'no-recurrence-events' accounts for roughly 70% of the total labels and 'recurrence-events' only accounts for 30%. As it doesn't have a lot training data (about 300 instances), the imbalanced class distribution might have a greater impact on the accuracy rate than on other datasets. 

